In [ ]:
import bats
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import persim
import plotly.express as px
import plotly.graph_objects as go
import ripser
import pickle
import tqdm
import collections
collections.Iterable = collections.abc.Iterable
from itertools import combinations
from natsort import natsorted
from persim import PersistenceImager
from scipy.spatial import distance_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold

In [ ]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
#START = 25
#END = 35
#STEP = 1

params = pd.read_csv('data/params_file.csv')
labels = params.end_has_niche

In [ ]:
i0 = params[params.id == 183].index[0]
i1 = params[params.id == 755].index[0]
i2 = params[params.id == 1189].index[0]

labels[i0], labels[i1], labels[i2]

In [ ]:
def rips_dgms(Id, celltype, START, END, STEP):
    files = natsorted(glob.glob(f'data/ID-{Id}_time-*_From*ParamSweep_Data.csv'))[START:END:STEP]

    bars = []
    for i,f in enumerate(files):
        df = pd.read_csv(f)
        mp = df[df.celltypes == celltype][['points_x', 'points_y']].to_numpy()
        if mp.size == 0: 
            continue
        else:
            dgms = ripser.ripser(mp, distance_matrix=False, maxdim=0)['dgms'][0]
            bars.extend( 
                dgms[dgms[:,1] != np.inf] 
                + [i, 0] 
            )

    return np.array(bars)


In [ ]:
def rips_dgms1(Id, celltype, START, END, STEP):
    files = natsorted(glob.glob(f'data/ID-{Id}_time-*_From*ParamSweep_Data.csv'))[START:END:STEP]

    bars = []
    for i,f in enumerate(files):
        df = pd.read_csv(f)
        mp = df[df.celltypes == celltype][['points_x', 'points_y']].to_numpy()
        if mp.size == 0: 
            continue
        else:
            dgms = ripser.ripser(mp, distance_matrix=False, maxdim=1)['dgms'][1]
            bars.extend( 
                dgms[dgms[:,1] != np.inf] 
                + [i, 0] 
            )

    return np.array(bars)

In [ ]:
def cell_count(Id, celltype, START, END, STEP):
    files = natsorted(glob.glob(f'data/ID-{Id}_time-*_From*ParamSweep_Data.csv'))[START:END:STEP]

    counts = []
    for i,f in enumerate(files):
        df = pd.read_csv(f)
        mp = df[df.celltypes == celltype]
        counts.append( len(mp) )

    return counts


In [ ]:
def cell_count_close(Id, celltype, START, END, STEP, r=2):
    files = natsorted(glob.glob(f'data/ID-{Id}_time-*_From*ParamSweep_Data.csv'))[START:END:STEP]
    assert len(files) > 0, f"{Id}"
    ds = []
    for i,f in enumerate(files):
        df = pd.read_csv(f)
        tum = df[df.celltypes == celltype][['points_x', 'points_y']].to_numpy()
        ves = df[df.celltypes == 'Vessel'][['points_x', 'points_y']].to_numpy()
        D = distance_matrix(tum, ves)
        if D.size == 0:
            ds.append( 0 )
        else:
            D[D == 0] = np.inf
            d = D.min(axis=1)
            ds.append( len(d[d <= r]) )
    return ds


In [ ]:
def vessel_proximity(Id, celltype, START, END, STEP):
    files = natsorted(glob.glob(f'data/ID-{Id}_time-*_From*ParamSweep_Data.csv'))[START:END:STEP]
    assert len(files) > 0, f"{Id}"
    ds = []
    for i,f in enumerate(files):
        df = pd.read_csv(f)
        tum = df[df.celltypes == celltype][['points_x', 'points_y']].to_numpy()
        ves = df[df.celltypes == 'Vessel'][['points_x', 'points_y']].to_numpy()
        D = distance_matrix(tum, ves)
        ds.append( np.sqrt(2)*50 if D.size == 0 else distance_matrix(tum, ves).min() )

    return ds


In [ ]:
def phenotype_ratio(Id, START, END, STEP):
    files = natsorted(glob.glob(f'data/ID-{Id}_time-*_From*ParamSweep_Data.csv'))[START:END:STEP]
    assert len(files) > 0, f"{Id}"
    rs = []
    for i,f in enumerate(files):
        df = pd.read_csv(f)
        p = df[df.celltypes == 'Macrophage'].phenotypes
        rs.append( 0 if p.size == 0 else len(p[p < .5]) / len(p) )

    return rs


In [ ]:
imgs = {}
scores = {}

In [ ]:
# with VR tumor dim 0 
# at time steps 250, 300, 350, 400, 450, 500
# 
# the following selects at 250
START = 25
END = 30
STEP = 5
Id = 1488
files = natsorted(glob.glob(f'data/ID-{Id}_time-*_From*ParamSweep_Data.csv'))[START:END:STEP]

Vietoris-Rips features

In [ ]:
tumor_VR0 = dict()
times = [25, 30, 35, 40, 45, 50]
for START in times:
    END = START + 5
    STEP = 5
    
    # compute Tumor VR 0
    diagrams_h0_rst = []
    empty_idx = []
    for i, ID in enumerate(tqdm.tqdm(params.id)):
        dgms = rips_dgms(ID, 'Tumour', START, END, STEP)
        if dgms.size == 0:
            empty_idx.append(i)
            dgms = np.array([ [0,1], [1,2] ])
        diagrams_h0_rst.append( dgms )

    pimgr_rst = PersistenceImager()
    imgs_rst = pimgr_rst.fit_transform(diagrams_h0_rst, skew=False)    
    imgs_array_rst = np.array([
        img.flatten()
        for img in imgs_rst
    ])
    for i in empty_idx:
        imgs_rst[i] = np.zeros(pimgr_rst.resolution)
        imgs_array_rst[i] = np.zeros(pimgr_rst.resolution).flatten()
    imgs['rst'] = imgs_rst

    clf = LogisticRegression(max_iter=1000)
    clf.fit(imgs_array_rst, labels)
    print(clf.score(imgs_array_rst, labels))

    cv = RepeatedStratifiedKFold()
    tumor_VR0[START] = cross_val_score(clf, imgs_array_rst, labels, cv=cv)
    

In [ ]:
tumor_VR1 = dict()
times = [25, 30, 35, 40, 45, 50]
for START in times:
    END = START + 5
    STEP = 5
    
    # compute Tumor VR 0
    diagrams_h1_rst = []
    empty_idx = []
    for i, ID in enumerate(tqdm.tqdm(params.id)):
        dgms = rips_dgms1(ID, 'Tumour', START, END, STEP)
        if dgms.size == 0:
            empty_idx.append(i)
            dgms = np.array([ [0,1], [1,2] ])
        diagrams_h1_rst.append( dgms )

    pimgr_rst = PersistenceImager()
    imgs_rst = pimgr_rst.fit_transform(diagrams_h1_rst, skew=False)    
    imgs_array_rst = np.array([
        img.flatten()
        for img in imgs_rst
    ])
    for i in empty_idx:
        imgs_rst[i] = np.zeros(pimgr_rst.resolution)
        imgs_array_rst[i] = np.zeros(pimgr_rst.resolution).flatten()
    imgs['rst'] = imgs_rst

    clf = LogisticRegression(max_iter=1000)
    clf.fit(imgs_array_rst, labels)
    print(clf.score(imgs_array_rst, labels))

    cv = RepeatedStratifiedKFold()
    tumor_VR1[START] = cross_val_score(clf, imgs_array_rst, labels, cv=cv)
    

In [228]:
macrophage_VR0 = dict()

times = [25, 30, 35, 40, 45, 50]
for START in times:
    END = START + 5
    STEP = 5
    
    # compute Tumor VR 0
    diagrams_h0_rst = []
    empty_idx = []
    for i, ID in enumerate(tqdm.tqdm(params.id)):
        dgms = rips_dgms(ID, 'Macrophage', START, END, STEP)
        if dgms.size == 0:
            empty_idx.append(i)
            dgms = np.array([ [0,1], [1,2] ])
        diagrams_h0_rst.append( dgms )

    pimgr_rst = PersistenceImager()
    pimgr_rst.fit(diagrams_h0_rst, skew=False)    
    pimgr_rst.birth_range = (0,1)
    imgs_rst = pimgr_rst.transform(diagrams_h0_rst, skew=False)    
    imgs_array_rst = np.array([
        img.flatten()
        for img in imgs_rst
    ])
    for i in empty_idx:
        imgs_rst[i] = np.zeros(pimgr_rst.resolution)
        imgs_array_rst[i] = np.zeros(pimgr_rst.resolution).flatten()
    imgs['rst'] = imgs_rst

    clf = LogisticRegression(max_iter=1000)
    clf.fit(imgs_array_rst, labels)
    print(clf.score(imgs_array_rst, labels))

    cv = RepeatedStratifiedKFold()
    macrophage_VR0[START] = cross_val_score(clf, imgs_array_rst, labels, cv=cv)
    

100%|█████████████████████████████| 1485/1485 [03:13<00:00,  7.66it/s]


0.8208754208754209


100%|█████████████████████████████| 1485/1485 [03:12<00:00,  7.73it/s]


0.8424242424242424


100%|█████████████████████████████| 1485/1485 [03:14<00:00,  7.62it/s]


...

0.8397306397306398


100%|█████████████████████████████| 1485/1485 [03:08<00:00,  7.86it/s]


0.8498316498316498


100%|█████████████████████████████| 1485/1485 [03:15<00:00,  7.60it/s]


0.8444444444444444


In [ ]:
macrophage_VR1 = dict()
times = [25, 30, 35, 40, 45, 50]
for START in times:
    END = START + 5
    STEP = 5
    
    # compute Tumor VR 0
    diagrams_h1_rst = []
    empty_idx = []
    for i, ID in enumerate(tqdm.tqdm(params.id)):
        dgms = rips_dgms1(ID, 'Macrophage', START, END, STEP)
        if dgms.size == 0:
            empty_idx.append(i)
            dgms = np.array([ [0,1], [1,2] ])
        diagrams_h1_rst.append( dgms )

    pimgr_rst = PersistenceImager()
    imgs_rst = pimgr_rst.fit_transform(diagrams_h1_rst, skew=False)    
    imgs_array_rst = np.array([
        img.flatten()
        for img in imgs_rst
    ])
    for i in empty_idx:
        imgs_rst[i] = np.zeros(pimgr_rst.resolution)
        imgs_array_rst[i] = np.zeros(pimgr_rst.resolution).flatten()
    imgs['rst'] = imgs_rst

    clf = LogisticRegression(max_iter=1000)
    clf.fit(imgs_array_rst, labels)
    print(clf.score(imgs_array_rst, labels))

    cv = RepeatedStratifiedKFold()
    macrophage_VR1[START] = cross_val_score(clf, imgs_array_rst, labels, cv=cv)

In [ ]:
# save data 
"""
f = open("analysis/tumor_VR0.pkl", "wb")
pickle.dump(tumor_VR0, f)
f.close()

f = open("analysis/tumor_VR1.pkl", "wb")
pickle.dump(tumor_VR1, f)
f.close()

f = open("analysis/macrophage_VR0.pkl", "wb")
pickle.dump(macrophage_VR0, f)
f.close()

f = open("analysis/macrophage_VR1.pkl", "wb")
pickle.dump(macrophage_VR1, f)
f.close()
"""

# Plot

In [ ]:
# open data

In [230]:
with open("analysis/tumor_VR0.pkl", 'rb') as f:
    tumor_VR0 = pickle.load(f)
    
with open("analysis/tumor_VR1.pkl", 'rb') as f:
    tumor_VR1 = pickle.load(f)
    
with open("analysis/macrophage_VR0.pkl", 'rb') as f:
    macrophage_VR0 = pickle.load(f)
    
with open("analysis/macrophage_VR1.pkl", 'rb') as f:
    macrophage_VR1 = pickle.load(f)

In [231]:
times = [25, 30, 35, 40, 45, 50]
x = [[t] * 50 for t in times]
x = [item for sublist in x for item in sublist]

y_tumor_VR0 = [tumor_VR0[t] for t in times]
y_tumor_VR0 = [item for sublist in y_tumor_VR0 for item in sublist]

y_tumor_VR1 = [tumor_VR1[t] for t in times]
y_tumor_VR1 = [item for sublist in y_tumor_VR1 for item in sublist]

y_macrophage_VR0 = [macrophage_VR0[t] for t in times]
y_macrophage_VR0 = [item for sublist in y_macrophage_VR0 for item in sublist]

y_macrophage_VR1 = [macrophage_VR1[t] for t in times]
y_macrophage_VR1 = [item for sublist in y_macrophage_VR1 for item in sublist]

In [233]:
#fig = go.Figure()

# tumor VR0
trace0 = go.Box(
        y = y_tumor_VR0,
        x = x,
        name = "tumour, dim 0",
)

# tumor VR1
trace1 = go.Box(
        y = y_tumor_VR1,
        x = x,
        name = "tumour, dim 1"
)

# macrophage VR0
trace2 = go.Box(
        y = y_macrophage_VR0,
        x = x,
        name = "macrophage, dim 0"
)

# macrophage VR1
trace3 = go.Box(
        y = y_macrophage_VR1,
        x = x,
        name = "macrophage, dim 1"
)


data = [trace0, trace1, trace2, trace3]
fig = go.Figure(data=data)

layout = go.Layout(
    yaxis=go.layout.YAxis(
        title='accuracy',
        titlefont = dict(size = 20),
        zeroline=False,
        tickfont = dict(size=20),
    ),
    xaxis = go.layout.XAxis(
        title="time",
        titlefont = dict(size = 20),
        tickfont = dict(size = 20)),
        boxmode='group',
)

fig = go.Figure(data = data, layout = layout)
fig.show()

#fig.write_image("analysis/VR_summary.png")
